In [1]:
import deepwalk as dw
import numpy as np
import pandas as pd

import networkx as nx

In [2]:
print('Uploading...')
ppi_adj = pd.read_csv('/root/work/data/stad_ppi', index_col=0)
exp_matrix = pd.read_csv('/root/work/data/stad_exp_matrix', index_col=0)
met_matrix = pd.read_csv('/root/work/data/stad_met_matrix', index_col=0)
label = pd.read_csv('/root/work/data/stad_label')
print('Uploading done!')

Uploading...
Uploading done!


In [7]:
exp_matrix

,TCGA-B7-5816,TCGA-B7-5818,TCGA-BR-6452,TCGA-BR-6453,TCGA-BR-6454,TCGA-BR-6455,TCGA-BR-6456,TCGA-BR-6457,TCGA-BR-6458,TCGA-BR-6563,...,TCGA-HU-A4H4,TCGA-HU-A4H5,TCGA-HU-A4H6,TCGA-HU-A4H8,TCGA-HU-A4HD,TCGA-IN-7806,TCGA-IN-7808,TCGA-IN-8462,TCGA-IN-8663,TCGA-IP-7968
Hugo_Symbol,,,,,,,,,,,,,,,,,,,,,
DOCK11,-0.172321,-1.698091,-0.395498,0.797247,0.040342,-0.405609,0.535314,0.777071,1.280067,0.847552,...,-0.558228,0.111199,-0.030437,-0.690486,-0.670120,-1.377293,2.053025,-0.365134,-1.958642,0.777071
GPBP1L1,-0.440377,-0.310255,0.000000,-0.290230,-0.690662,0.180130,-0.290230,-0.750740,-0.530477,-0.110078,...,0.020023,-0.280222,-0.060042,0.480312,0.170121,0.530336,0.340228,-0.270218,0.250180,0.590366
MLYCD,-0.612400,-3.361548,-0.411500,-0.210758,-0.170581,-1.387321,-0.863770,-0.401467,0.030055,-0.270945,...,-1.105208,-1.427725,-0.763211,-0.552119,-0.642551,-0.421543,-0.632549,-1.256274,-0.612400,-0.753107
GMPS,1.511755,0.881231,1.331625,0.320520,1.041395,1.311616,1.521761,0.010000,1.231557,0.250431,...,2.102079,0.741088,0.941298,0.991343,1.461725,0.320520,0.120206,0.180304,0.871218,1.071420
CYBA,1.400185,1.740214,0.850124,2.320249,2.010232,0.590084,0.910132,0.970139,1.340179,1.650207,...,1.330178,-0.380141,1.080152,-0.150075,0.220014,0.119994,0.980140,0.740108,0.380048,0.089988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOXF2,-2.536788,-6.539229,-1.355439,-1.194522,-1.738168,-2.953452,0.180372,0.250515,-0.070207,0.020030,...,-1.970231,-3.877121,-0.882986,-2.709256,-1.566868,-1.325294,-4.929512,-0.070207,-1.929779,-1.003541
AADAT,-0.126643,-0.441247,-0.248364,-2.837900,0.671308,0.055521,0.711658,-0.086208,1.234867,-1.382676,...,0.499929,0.207289,-0.777358,-2.968549,0.812388,-0.086208,-0.818051,-0.065950,-2.216349,0.913050
FRMD4B,-0.814555,0.471615,-0.583006,-0.311501,0.712296,-0.583006,-0.794463,-0.231112,-0.231112,-0.020106,...,-0.643442,0.281041,-0.532735,0.130491,-0.020106,-1.531254,-0.351671,1.503851,-0.532735,0.080272


In [3]:
# construct a graph
ppi_graph = nx.from_pandas_adjacency(ppi_adj)
len(ppi_graph)
## 5s

9742

In [4]:
label_sp = label[0:10]
embeddings = np.empty
for i in range(len(label_sp)):
    print('No.',i)
    exp = pd.DataFrame(exp_matrix.iloc[:,i].T)
    met = pd.DataFrame(met_matrix.iloc[:,i].T)
    feature = met.join(exp, lsuffix='_met', rsuffix='_exp').to_dict('index')
    nx.set_node_attributes(ppi_graph, feature)
    emb_dict = dw.getEmbedding(ppi_graph)
    emb = np.array(pd.DataFrame.from_dict(emb_dict)).reshape(1,32*ppi_adj.shape[0])
    if i == 0:
        embeddings = emb
    else:
        embeddings = np.concatenate((embeddings, emb), axis=0)

No. 0


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:   12.1s remaining:   36.4s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   31.8s finished


Learning embedding vectors...
Learning embedding vectors done!
No. 1


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:   11.4s remaining:   34.3s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   31.4s finished


Learning embedding vectors...
Learning embedding vectors done!
No. 2


...

Learning embedding vectors...
Learning embedding vectors done!
No. 8


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:   11.8s remaining:   35.3s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   33.4s finished


Learning embedding vectors...
Learning embedding vectors done!
No. 9


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:   12.2s remaining:   36.6s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   34.4s finished


Learning embedding vectors...
Learning embedding vectors done!


In [5]:
np.save('/root/work/data/emb_stad_32_10.npy',embeddings)
print('Saving done!')

Saving done!


In [6]:
# #single test
# met = pd.DataFrame(met_matrix.iloc[:,0].T)
# exp = pd.DataFrame(exp_matrix.iloc[:,0].T)
# feature = met.join(exp, lsuffix='_met', rsuffix='_exp').to_dict('index')
# nx.set_node_attributes(ppi_graph, feature)
# emb_dict = dw.getEmbedding(ppi_graph)
# ## 2m43s    
# emb1 = np.array(pd.DataFrame.from_dict(emb_dict)).reshape(1,64*ppi_adj.shape[0])